In [1]:
import os
from dotenv import load_dotenv
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.chains.llm import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv


In [2]:
# Selecting LLM Model from HuggingFace 
REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"

# Getting HF_TOKEN
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN_CALVIN")

In [3]:
hf = HuggingFaceHub(
    huggingfacehub_api_token = HF_TOKEN,
    repo_id=REPO_ID,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 500,
        "top_k": 30,
        "max_length": 400,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

# hf = HuggingFacePipeline.from_model_id(
#     model_id=REPO_ID,
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 200},
# )

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [76]:
prompt = PromptTemplate.from_template(
  """You are a professional and friendly Bank Customer Service and you are helping a client with bank knowledge. If a client is asking about general question, answer it with human-like response. When a client asking an issue related to banks Just explain it in detail. This is the issue: {issue} 
    If the issue is related to the Bank, you need to know the following information will help solve the client's issue, this is the information: {knowledge} 

    If the issue is not related to bank, just answer that you not capable of answering the client's issue. Dont give a misleading information to the client

    ANSWER :
    """
)

In [77]:
llm_chain = LLMChain(prompt=prompt, llm=hf)

In [79]:
user_input = "what is computational physics?"

# Vectara

In [80]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Vectara
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
from langchain.prompts import SystemMessagePromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.combine_documents.reduce import ReduceDocumentsChain
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain_text_splitters.character import CharacterTextSplitter
from langchain import hub

In [81]:
VECTARA_CUSTOMER_ID = os.getenv("VECTARA_CUSTOMER_ID")
VECTARA_CORPUS_ID = os.getenv("VECTARA_CORPUS_ID")
VECTARA_API_KEY = os.getenv("VECTARA_API_KEY")

vectara_client = Vectara(
                vectara_customer_id=VECTARA_CUSTOMER_ID,
                vectara_corpus_id=VECTARA_CORPUS_ID,
                vectara_api_key=VECTARA_API_KEY
)

In [82]:
def get_knowledge_content(vectara, query, threshold=0.5):
    found_docs = vectara.similarity_search_with_score(
        query,
        score_threshold=threshold,
    )
    # print(len(found_docs))
    # knowledge_content = ""
    # for number, (score, doc) in enumerate(found_docs):
    #     knowledge_content += f"\n{found_docs[number][0].page_content}\n"
    # return knowledge_content
    return found_docs

In [83]:
knowledge_content = get_knowledge_content(vectara_client, user_input)

In [84]:
map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=hf, prompt=map_prompt)
summarize = map_chain.invoke({"docs":knowledge_content})

In [85]:
print(summarize['text'])


Human: The following is a set of documents:
[(Document(page_content='When we may make changes\nThe circumstances in which we may make changes to\nyour contract include but are not limited to:\na)  when the cost of providing credit to you changes;\n\n                                                  17 Terms and\nconditions b)  as necessary or desirable to comply with or reflect\n         any law, code of conduct, regulator guidance or\n         requirement, or decision of a court or other dispute\n       resolution process;\nc)  to reflect changes in technology or our processes,\n        including computer systems;\nd)  to include new product features or services or to\n        ensure that the terms and conditions match our\n       operational processes;\ne)  to discontinue or replace a product, and for this\n        purpose we may change your product to a different\n         product with similar features to the discontinued or\n       replaced product;\nf)  to consolidate or simplify 

# RAG Chaining

In [86]:
runnable = llm_chain

In [87]:
response = runnable.invoke({"knowledge": summarize['text'],"issue": user_input})

In [88]:
print(response)

{'knowledge': "Human: The following is a set of documents:\n[(Document(page_content='When we may make changes\\nThe circumstances in which we may make changes to\\nyour contract include but are not limited to:\\na)  when the cost of providing credit to you changes;\\n\\n                                                  17 Terms and\\nconditions b)  as necessary or desirable to comply with or reflect\\n         any law, code of conduct, regulator guidance or\\n         requirement, or decision of a court or other dispute\\n       resolution process;\\nc)  to reflect changes in technology or our processes,\\n        including computer systems;\\nd)  to include new product features or services or to\\n        ensure that the terms and conditions match our\\n       operational processes;\\ne)  to discontinue or replace a product, and for this\\n        purpose we may change your product to a different\\n         product with similar features to the discontinued or\\n       replaced product

In [91]:
str_response = response['text']

answer_part = str_response.split("ANSWER :", 1)[1]

# Optional: Remove any leading or trailing whitespace
answer_part = answer_part.strip()

print(answer_part)

I'm here to help answer any questions you might have related to banking and our services. However, I'm unable to provide an answer to the question "what is computational physics?" as it is not related to banking or our services. If you have any questions about your account, loan, credit card, or any other banking-related matter, please let me know and I'll be happy to help.


In [ ]:
print(response['text'])

You are a professional and friendly Bank Customer Service and you are helping a client with bank knowledge. If a client is asking about general question, answer it with human-like response. When a client asking an issue related to banks Just explain it in detail. This is the issue: what is computational physics? 
    If the issue is related to the Bank, you need to know the following information will help solve the client's issue, this is the information: Human: The following is a set of documents:
[(Document(page_content='When we may make changes\nThe circumstances in which we may make changes to\nyour contract include but are not limited to:\na)  when the cost of providing credit to you changes;\n\n                                                  17 Terms and\nconditions b)  as necessary or desirable to comply with or reflect\n         any law, code of conduct, regulator guidance or\n         requirement, or decision of a court or other dispute\n       resolution process;\nc)  to re